# PyCity Schools Analysis

## Three observation trends based on the data

1. [School Type] Whole average values are not so different(less than 1 point/ 1%) but 5 bottom performing schools (by passing rate) are all Charter.  
2. [Spending Ranges Per Student] No positive correlation between Spending Ranges Per Student and Passing Rates, rather than there might be negative correlation.
3. [School Size] Smaller schools likely to have higher average or rate than larger ones.

In [135]:
# Import Dependencies
import pandas as pd
import os
import numpy as np

# Path to resource files
path = "resources"

# Genarate a student df
students_df = pd.read_csv(os.path.join(path, 'students_complete.csv'))

# Generate a school df changed the column name 'name' to 'school'
school_df = pd.read_csv(os.path.join(path, 'schools_complete.csv')).rename(columns={"name": "school"})

# Merge two dfs into one df
stu_sch_df = pd.merge(students_df, school_df, how = "outer")
# stu_sch_df.head()

## District Summary

In [136]:
# generate a new dataframe for schools "dist_sch_df" in whicn rows "type" is "District" are collected
dist_sch_df = school_df[school_df["type"] == "District"]

# generate a new dataframe for students "dist_stu_sch_df" in whicn rows "type" is "District" are collected
dist_stu_sch_df = stu_sch_df[stu_sch_df["type"] == "District"]

# group "dist_stu_sch_df" (for students) by type
distByType = stu_sch_df.groupby("type")

# assign the computed numbers to the fields
totalStu = len(dist_stu_sch_df.index)
math_passed = len(dist_stu_sch_df[dist_stu_sch_df["math_score"] >= 60].index)
read_passed = len(dist_stu_sch_df[dist_stu_sch_df["reading_score"] >= 60].index)

# create a new df with computed data
dist_summary = pd.DataFrame({
    "Total Schools":[len(dist_sch_df.index)],
    "Total Students":[totalStu],
    "Total Budget":[distByType.sum().loc["District","budget"]],
    "Average Math Score":[distByType.mean().loc["District","math_score"]],
    "Average Reading Score":[distByType.mean().loc["District","reading_score"]],
    "% Passing Math":[math_passed/totalStu],
    "% Passing Reading":[read_passed/totalStu],
    "% Overall Passing Rate":[(math_passed + read_passed)/(totalStu*2)]
})

# format and show the df
dist_summary[[
    "Total Schools",
    "Total Students",
    "Total Budget",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
]].style.format({
    "Total Budget":"${:,d}",
    "Average Math Score":"{:.2f}",
    "Average Reading Score":"{:.2f}",
    "% Passing Math":"{:.2%}",
    "% Passing Reading":"{:.2%}",
    "% Overall Passing Rate":"{:.2%}"
})


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,7,26976,"$70,439,053,973",76.99,80.96,89.03%,100.00%,94.52%


## School Summary

In [137]:
# generate a new df for students grouped by "school"
groupBySch = stu_sch_df.groupby("school")

# assign the computed numbers to the fields
passedStu_mathbySch = stu_sch_df[stu_sch_df["math_score"] >= 60].groupby("school").count()["Student ID"]
passedStu_readbySch = stu_sch_df[stu_sch_df["reading_score"] >= 60].groupby("school").count()["Student ID"]

# generate a new df to show the results
sch_summ_df = groupBySch.mean()

# assign computed data to sch_summ_df
sch_summ_df["School Type"] = list(school_df["type"])
sch_summ_df["Total Students"] = groupBySch.count()["Student ID"]
sch_summ_df["Per Student Budget"] = sch_summ_df["budget"] / sch_summ_df["Total Students"]
sch_summ_df["Average Math Score"] = groupBySch.mean()["math_score"]
sch_summ_df["Average Reading Score"] = groupBySch.mean()["reading_score"]
sch_summ_df["% Passing Math"] = passedStu_mathbySch / sch_summ_df["Total Students"]
sch_summ_df["% Passing Reading"] = passedStu_readbySch / sch_summ_df["Total Students"]
sch_summ_df["% Overall Passing Rate"] = (passedStu_mathbySch + passedStu_readbySch)/(sch_summ_df["Total Students"]*2)

# format and show the result
sch_summ = sch_summ_df.rename(columns={"budget": "Total School Budget"})[[
    "School Type",
    "Total Students",
    "Total School Budget",
    "Per Student Budget",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
    
]]
sch_summ.style.format({
    "Total School Budget":"${:,.0f}",
    "Per Student Budget":"${:,.0f}",
    "Average Math Score":"{:.2f}",
    "Average Reading Score":"{:.2f}",
    "% Passing Math":"{:.2%}",
    "% Passing Reading":"{:.2%}",
    "% Overall Passing Rate":"{:.2%}"
    
})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,,,,,,,,,
Bailey High School,District,4976,"$3,124,928",$628,77.05,81.03,89.53%,100.00%,94.76%
Cabrera High School,District,1858,"$1,081,356",$582,83.06,83.98,100.00%,100.00%,100.00%
Figueroa High School,Charter,2949,"$1,884,411",$639,76.71,81.16,88.44%,100.00%,94.22%
Ford High School,District,2739,"$1,763,916",$644,77.10,80.75,89.30%,100.00%,94.65%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,100.00%,100.00%,100.00%
Hernandez High School,Charter,4635,"$3,022,020",$652,77.29,80.93,89.08%,100.00%,94.54%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,100.00%,100.00%,100.00%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,88.86%,100.00%,94.43%
Johnson High School,Charter,4761,"$3,094,650",$650,77.07,80.97,89.18%,100.00%,94.59%


## Top Performing Schools (By Passing Rate)

In [138]:
# sort, format, and show the df
sch_summ.sort_values(by=["% Overall Passing Rate", "% Passing Math"], ascending=False).head().style.format({
    "Total School Budget":"${:,.0f}",
    "Per Student Budget":"${:,.0f}",
    "Average Math Score":"{:.2f}",
    "Average Reading Score":"{:.2f}",
    "% Passing Math":"{:.2%}",
    "% Passing Reading":"{:.2%}",
    "% Overall Passing Rate":"{:.2%}"
    
})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,,,,,,,,,
Cabrera High School,District,1858,"$1,081,356",$582,83.06,83.98,100.00%,100.00%,100.00%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,100.00%,100.00%,100.00%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,100.00%,100.00%,100.00%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,100.00%,100.00%,100.00%
Shelton High School,District,1761,"$1,056,600",$600,83.36,83.73,100.00%,100.00%,100.00%


## Bottom Performing Schools (By Passing Rate)

In [139]:
# sort, format, and show the df
sch_summ.sort_values(by=["% Overall Passing Rate", "% Passing Math"]).head().style.format({
    "Total School Budget":"${:,.0f}",
    "Per Student Budget":"${:,.0f}",
    "Average Math Score":"{:.2f}",
    "Average Reading Score":"{:.2f}",
    "% Passing Math":"{:.2%}",
    "% Passing Reading":"{:.2%}",
    "% Overall Passing Rate":"{:.2%}"
    
})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,,,,,,,,,
Figueroa High School,Charter,2949,"$1,884,411",$639,76.71,81.16,88.44%,100.00%,94.22%
Rodriguez High School,Charter,3999,"$2,547,363",$637,76.84,80.74,88.55%,100.00%,94.27%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,88.86%,100.00%,94.43%
Hernandez High School,Charter,4635,"$3,022,020",$652,77.29,80.93,89.08%,100.00%,94.54%
Johnson High School,Charter,4761,"$3,094,650",$650,77.07,80.97,89.18%,100.00%,94.59%


## Math Scores by Grade

In [140]:
# generate a new df to show the result
mathScore_sch_grade = pd.DataFrame({
    "9th":stu_sch_df[stu_sch_df["grade"] == "9th"].groupby("school").mean()["math_score"],
    "10th":stu_sch_df[stu_sch_df["grade"] == "10th"].groupby("school").mean()["math_score"],
    "11th":stu_sch_df[stu_sch_df["grade"] == "11th"].groupby("school").mean()["math_score"],
    "12th":stu_sch_df[stu_sch_df["grade"] == "12th"].groupby("school").mean()["math_score"]
})

# format and show the df
mathScore_sch_grade[[
    "9th", "10th", "11th", "12th"
]].style.format("{:.2f}")

,9th,10th,11th,12th
school,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [141]:
# generate a new df to show the result
readScore_sch_grade = pd.DataFrame({
    "9th":stu_sch_df[stu_sch_df["grade"] == "9th"].groupby("school").mean()["reading_score"],
    "10th":stu_sch_df[stu_sch_df["grade"] == "10th"].groupby("school").mean()["reading_score"],
    "11th":stu_sch_df[stu_sch_df["grade"] == "11th"].groupby("school").mean()["reading_score"],
    "12th":stu_sch_df[stu_sch_df["grade"] == "12th"].groupby("school").mean()["reading_score"]
})

# format and show the df
readScore_sch_grade[[
    "9th", "10th", "11th", "12th"
]].style.format("{:.2f}")


,9th,10th,11th,12th
school,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [142]:
# assign bins and labels
bins_spend = [0,585,615,645,675]
group_labels_spend = ["< $585","$585 - $615","$615 - $645","$645 - $675"]

# assign a new column as labels
sch_summ["Spending Ranges (Per Student)"] = pd.cut(sch_summ["Per Student Budget"],bins_spend,labels=group_labels_spend)

# Group by ageRange
sch_spendRange = sch_summ.groupby("Spending Ranges (Per Student)").mean()

# format and show the df
sch_spendRange[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
]].style.format({
    "Average Math Score": "{:.2f}",
    "Average Reading Score": "{:.2f}",
    "% Passing Math": "{:.2%}",
    "% Passing Reading": "{:.2%}",
    "% Overall Passing Rate":"{:.2%}"   
})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
< $585,83.46,83.93,100.00%,100.00%,100.00%
$585 - $615,83.60,83.89,100.00%,100.00%,100.00%
$615 - $645,79.08,81.89,92.64%,100.00%,96.32%
$645 - $675,77.00,81.03,89.04%,100.00%,94.52%


In [143]:
# assign bins and labels
bins_size = [0,1000,2000,5000]
group_labels_size = ["Small(< 1000)","Medium(1000-2000)","Large(2000-5000)"]

# assign a new column as labels
sch_summ["School Size"] = pd.cut(sch_summ["Total Students"],bins_size,labels=group_labels_size)

# Group by sizeRange
sch_sizeRange = sch_summ.groupby("School Size").mean()

# format and show the df
sch_sizeRange[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
]].style.format({
    "Average Math Score": "{:.2f}",
    "Average Reading Score": "{:.2f}",
    "% Passing Math": "{:.2%}",
    "% Passing Reading": "{:.2%}",
    "% Overall Passing Rate":"{:.2%}"   
})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small(< 1000),83.82,83.93,100.00%,100.00%,100.00%
Medium(1000-2000),83.37,83.86,100.00%,100.00%,100.00%
Large(2000-5000),77.75,81.34,90.37%,100.00%,95.18%


In [144]:
# group by school type, format, and show the df
sch_summ.groupby("School Type").mean()[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
]].style.format({
    "Average Math Score": "{:.2f}",
    "Average Reading Score": "{:.2f}",
    "% Passing Math": "{:.2%}",
    "% Passing Reading": "{:.2%}",
    "% Overall Passing Rate":"{:.2%}"   
})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,80.32,82.43,94.41%,100.00%,97.20%
District,80.56,82.64,95.38%,100.00%,97.69%
